# Smart Shopping Assistant

Author: Xiaohui Chen

## Overview

This notebook demonstrates a **multi-agent shopping assistant** built with the **OpenAI Agents SDK**. The system helps users find, compare, and research products using specialized AI agents that collaborate through **tools** and **handoffs**.

### Architecture

```
                              ┌─────────────────────┐
                              │        USER         │
                              └──────────┬──────────┘
                                         │ ①
                                         ▼
                              ┌─────────────────────┐
                              │  🎯 CONCIERGE AGENT │◄─────────────────────┐
                              │    (Orchestrator)   │                      │
                              └──────────┬──────────┘                      │
                                         │ ②                               │
                           ┌─────────────┴─────────────┐                   │
                           │         HANDOFF           │                   │
                           ▼                           ▼                   │
                ┌─────────────────────┐     ┌─────────────────────┐        │
                │ 🔍 PRODUCT          │     │ ⭐ REVIEW           │        │
                │    SPECIALIST       │     │    ANALYST          │        │ ③
                │                     │     │                     │        │
                │ • Search products   │     │ • Fetch reviews     │        │
                │ • Compare prices    │     │ • Analyze sentiment │        │
                │ • Get specs         │     │ • Summarize pros/   │        │
                │                     │     │   cons              │        │
                └──────────┬──────────┘     └──────────┬──────────┘        │
                           │                           │                   │
                           └───────────────────────────┘                   │
                                         │                                 │
                                         └───── HANDOFF BACK ──────────────┘
                                                     │ ④
                                                     ▼
                                         ┌─────────────────────┐
                                         │       USER          │
                                         │    (Response)       │
                                         └─────────────────────┘
```

**Flow:**
1. User sends a request to the Concierge
2. Concierge analyzes intent and hands off to the appropriate specialist
3. Specialist uses tools to complete the task and hands back to Concierge
4. Concierge synthesizes the response and replies to the user

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Tools** | Functions that agents can call to interact with external APIs (e.g., product search, review fetching) |
| **Handoffs** | Mechanism for one agent to delegate tasks to another specialized agent |
| **Orchestration** | The Concierge agent routes requests to specialists and synthesizes their responses |

### Data Source

All product data is retrieved from the **Real-Time Product Search API** (via RapidAPI), which aggregates results from Google Shopping across multiple retailers including Amazon, Best Buy, Walmart, and more.


## 🎯 Concierge Agent - Low Level Design

### Model Selection

| Attribute | Value |
|-----------|-------|
| **Model** | `gpt-5.2-pro` |
| **Context Window** | 400,000 tokens |
| **Max Output** | 128,000 tokens |
| **Knowledge Cutoff** | August 31, 2025 |
| **Reasoning Effort** | `medium`, `high`, `xhigh` |
| **Input Modalities** | Text, Images |
| **Output Modalities** | Text |

**Supported Features:**

| Feature | Supported |
|---------|-----------|
| Function Calling | ✅ Yes |
| Streaming | ✅ Yes |
| Web Search | ✅ Yes |
| File Search | ✅ Yes |
| MCP (Model Context Protocol) | ✅ Yes |
| Structured Outputs | ❌ No |
| Fine-Tuning | ❌ No |
| Code Interpreter | ❌ No |

**API Endpoint:** `v1/responses`

**Why `gpt-5.2-pro` for Concierge?**

The model is selected for the Concierge Agent due to its superior reasoning capabilities:
- **Complex Reasoning** - Supports `xhigh` reasoning effort for nuanced intent classification
- **Large Context** - 400K tokens allows maintaining full conversation history
- **Function Calling** - Essential for triggering handoffs to specialist agents
- **MCP Support** - Enables integration with external tool servers

### Responsibilities

**Pre-Handoff Phase** (Initial user request):

1. **Receive User Input** - Accept the user's message
2. **Understand Intent** - Determine what the user wants (search, compare, review analysis, etc.)
3. **Decide Routing** - Choose to hand off to a specialist or respond directly
4. **Execute Handoff** - Call the appropriate transfer function (SDK-generated)

**Post-Handoff Phase** (After receiving results from specialists):

1. **Receive Specialist Results** - Accept the handoff back with task results
2. **Synthesize Response** - Format results into a user-friendly response
3. **Offer Next Steps** - Suggest related actions (e.g., "Would you like to see reviews?")
4. **Handle Follow-ups** - Route additional requests or answer questions about results

### Native Handoffs (OpenAI Agents SDK)

The Concierge Agent uses the **native `handoffs` parameter** from the OpenAI Agents SDK. This is the idiomatic way to implement agent-to-agent transfers.

**Bidirectional Handoff Configuration:**

For the Concierge to receive control back from specialists, we use **bidirectional handoffs** - each agent can transfer to the other.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           BIDIRECTIONAL HANDOFFS                            │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│                         ┌─────────────────────┐                             │
│                         │     CONCIERGE       │                             │
│                         │                     │                             │
│                         │ handoffs=[          │                             │
│                         │   product_specialist│                             │
│                         │   review_analyst    │                             │
│                         │ ]                   │                             │
│                         └──────────┬──────────┘                             │
│                                    │                                        │
│              transfer_to_          │           transfer_to_                 │
│              product_specialist    │           review_analyst               │
│                    │               │               │                        │
│                    ▼               │               ▼                        │
│     ┌─────────────────────┐       │       ┌─────────────────────┐          │
│     │ PRODUCT SPECIALIST  │       │       │   REVIEW ANALYST    │          │
│     │                     │       │       │                     │          │
│     │ handoffs=[concierge]│       │       │ handoffs=[concierge]│          │
│     └──────────┬──────────┘       │       └──────────┬──────────┘          │
│                │                  │                  │                      │
│                └──────────────────┼──────────────────┘                      │
│                                   │                                         │
│                    transfer_to_concierge                                    │
│                                   │                                         │
│                                   ▼                                         │
│                         ┌─────────────────────┐                             │
│                         │     CONCIERGE       │                             │
│                         │   (receives results)│                             │
│                         └─────────────────────┘                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

**Transfer Tools Created by SDK:**

| Agent | Has Transfer Tool | Can Transfer To |
|-------|-------------------|-----------------|
| **Concierge** | `transfer_to_product_specialist` | Product Specialist |
| **Concierge** | `transfer_to_review_analyst` | Review Analyst |
| **Product Specialist** | `transfer_to_concierge` | Concierge |
| **Review Analyst** | `transfer_to_concierge` | Concierge |

**Handoff Descriptions:**

| From | To | Description |
|------|-----|-------------|
| Concierge | Product Specialist | Search for products, compare prices, get specifications |
| Concierge | Review Analyst | Analyze product reviews, summarize customer feedback |
| Product Specialist | Concierge | Return results after completing product search/details task |
| Review Analyst | Concierge | Return results after completing review analysis task |

**Benefits of Bidirectional Handoffs:**
- ✅ Full orchestration control - Concierge can synthesize final responses
- ✅ Supports multi-step workflows (e.g., search → review → recommendation)
- ✅ Conversation context preserved throughout the cycle
- ✅ Specialists focus on tasks, Concierge handles user communication

### Routing Logic

| User Request Type | Action | Transfer Tool |
|-------------------|--------|---------------|
| Product search ("find me a laptop") | Hand off | `transfer_to_product_specialist` |
| Price comparison ("best deal") | Hand off | `transfer_to_product_specialist` |
| Specifications ("what's the processor") | Hand off | `transfer_to_product_specialist` |
| Review analysis ("what do people say") | Hand off | `transfer_to_review_analyst` |
| Sentiment questions ("any complaints") | Hand off | `transfer_to_review_analyst` |
| General questions ("what can you do") | Respond directly | None |
| Ambiguous requests ("I need something") | Ask clarification | None |

### System Prompt

```
You are the Concierge Agent for a Smart Shopping Assistant. Your role is to understand 
user requests, route them to specialist agents, and synthesize their results for the user.

## Your Responsibilities

1. GREET users and understand their shopping needs
2. ROUTE requests to the appropriate specialist by calling the transfer function
3. RECEIVE results back from specialists and present them to the user
4. OFFER next steps and handle follow-up requests
5. HANDLE general questions about your capabilities directly
6. ASK for clarification when requests are too vague

## Available Specialists

You have access to two specialist agents via transfer functions:

### Product Specialist (transfer_to_product_specialist)
Transfer to this agent when user wants to:
- Search for products ("find me a laptop", "show me TVs under $500")
- Compare prices across stores ("where's the best deal", "cheapest option")
- Get product specifications ("what's the processor", "specs for MacBook")
- Filter or sort products ("highest rated gaming laptops")

The Product Specialist has access to product search and details tools.

### Review Analyst (transfer_to_review_analyst)
Transfer to this agent when user wants to:
- Read or understand reviews ("what do people say about...")
- Analyze product quality ("is this reliable", "any complaints")
- Understand pros and cons ("what's good/bad about...")
- See rating breakdowns ("how many 5-star reviews")

The Review Analyst has access to review fetching and sentiment analysis tools.

## When NOT to Transfer

Handle these yourself (do not transfer):
- General questions: "What can you help me with?", "How does this work?"
- Greetings: "Hi", "Hello"
- Vague requests that need clarification first
- Follow-up questions that don't need specialist tools

## When Receiving Results from Specialists

After a specialist completes their task and transfers back to you:

1. PRESENT the results in a clear, user-friendly format
2. HIGHLIGHT key information (best prices, top-rated products, main pros/cons)
3. OFFER relevant next steps:
   - After product search: "Would you like to see reviews for any of these?"
   - After review analysis: "Would you like to search for alternatives?"
   - After price comparison: "Would you like more details on the cheapest option?"
4. BE READY to route follow-up requests to the appropriate specialist

## Clarification Guidelines

If a request is too vague to route, ask for clarification. Examples:
- "I need something" → Ask what type of product and budget
- "Help me shop" → Ask what they're looking for
- "What's good?" → Ask what category they're interested in

Always be friendly and helpful when asking for more information.

## Examples

### Initial Routing

User: "Find me a gaming laptop under $1500"
→ Transfer to Product Specialist (product search request with clear criteria)

User: "What do reviews say about the MacBook Air battery?"
→ Transfer to Review Analyst (review analysis with specific focus area)

User: "Where can I get the best price on AirPods Pro?"
→ Transfer to Product Specialist (price comparison request)

User: "Is the Samsung S24 reliable?"
→ Transfer to Review Analyst (quality/reliability question)

User: "I need a new phone"
→ Respond directly: Ask clarifying questions about budget and preferences

### After Receiving Results

[After Product Specialist returns with laptop search results]
→ Present the top options with prices and key specs
→ Offer: "Would you like me to check the reviews for any of these options?"

[After Review Analyst returns with MacBook Air review summary]
→ Present the pros, cons, and overall sentiment
→ Offer: "Would you like to compare prices or see alternatives?"
```

### Handoff Flow Diagram (Complete Cycle)

```
┌─────────────────────────────────────────────────────────────────┐
│                        USER MESSAGE                              │
│         "Find me a gaming laptop under $1500"                   │
└─────────────────────────────────────────────────────────────────┘
                               │
                               │ ① User Request
                               ▼
                    ┌─────────────────────┐
                    │   CONCIERGE AGENT   │
                    │   (gpt-5.2-pro)     │
                    │                     │
                    │   Handoffs:         │
                    │   • product_        │
                    │     specialist      │
                    │   • review_analyst  │
                    └──────────┬──────────┘
                               │
                               │ ② Decides to transfer
                               │   (product search request)
                               ▼
                    ┌─────────────────────┐
                    │   CALLS:            │
                    │   transfer_to_      │
                    │   product_specialist│
                    └──────────┬──────────┘
                               │
                               │ ③ SDK transfers control
                               ▼
                    ┌─────────────────────┐
                    │ PRODUCT SPECIALIST  │
                    │                     │
                    │   Tools:            │
                    │   • search_products │
                    │   • get_product_    │
                    │     details         │
                    │                     │
                    │   Handoffs:         │
                    │   • concierge       │
                    └──────────┬──────────┘
                               │
                               │ ④ Executes tools, gets results
                               ▼
                    ┌─────────────────────┐
                    │   CALLS:            │
                    │   search_products(  │
                    │     "gaming laptop",│
                    │     max_price=1500  │
                    │   )                 │
                    │                     │
                    │   Returns: 5 laptops│
                    │   with prices       │
                    └──────────┬──────────┘
                               │
                               │ ⑤ Task complete, transfer back
                               ▼
                    ┌─────────────────────┐
                    │   CALLS:            │
                    │   transfer_to_      │
                    │   concierge         │
                    │                     │
                    │   (with results in  │
                    │    conversation     │
                    │    context)         │
                    └──────────┬──────────┘
                               │
                               │ ⑥ SDK transfers control back
                               ▼
                    ┌─────────────────────┐
                    │   CONCIERGE AGENT   │
                    │   (receives results)│
                    │                     │
                    │   • Synthesizes     │
                    │     response        │
                    │   • Formats for     │
                    │     user            │
                    │   • Offers next     │
                    │     steps           │
                    └──────────┬──────────┘
                               │
                               │ ⑦ Final response
                               ▼
                    ┌─────────────────────┐
                    │   USER RESPONSE     │
                    │                     │
                    │   "Here are 5       │
                    │   gaming laptops    │
                    │   under $1500:      │
                    │   1. ASUS ROG...    │
                    │   2. MSI Stealth... │
                    │   ...               │
                    │                     │
                    │   Would you like to │
                    │   see reviews for   │
                    │   any of these?"    │
                    └─────────────────────┘
```
